In [1]:
import torch
import seaborn as sns
import sys
from pathlib import Path
sys.path.insert(0, "../..")
from src.data import make_dataset
from src.models import imagemodels
from src.models import train_model
import gin

/home/azureuser/.cache/pypoetry/virtualenvs/poetry_add_pandas-Tn0MmM9i-py3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-06 11:04:52.449946: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 11:04:52.631159: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-06 11:04:52.631185: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore abov

In [2]:
gin.parse_config_file("model.gin")
train_dataloader, test_dataloader = make_dataset.get_MNIST()
# print(gin.config_str())

A big advantage is that we can save this config as a file; that way it is easy to track what you changed during your experiments.

In [3]:
import torch.optim as optim
from src.models import metrics
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [7]:
import torch.optim as optim
gin.parse_config_file("model.gin")


learning_rates = [0.001] #[0.005, 0.001, 0.0005]
kernel_sizes = [3] #[1,3,6,12] #default: 3
filters1 = [64]#[16,32,64] #[24,28,32,40,48]#[16,32,64,128] #default: 32
filters2 = [32]#[16,32,64] #[24,28,32,40,48]#[16,32,64,128] #default: 32
units1 = [254] #[48,56,64,78,92] #[16,32,64,128]#default: 64
units2 = [128]#[16,32,64] #[20,32,56] #[16,32,64,128]#default: 32
epoch = 10
lr = 0.001
num_classes = 10



gin.bind_parameter("trainloop.log_dir", "../../models/units_padding_all1_kernal3_l288/")

gin.bind_parameter("CNN_padding1.num_classes", num_classes)
# gin.bind_parameter("CNN.kernel_size", kernel_size)
gin.bind_parameter("trainloop.epochs", epoch)



for fl1 in filters1:
    for fl2 in filters2:
        if fl2 <= fl1:
            for ut1 in units1:
                for ut2 in units2:
                    if ut2 <= ut1:
                        for lr in learning_rates:
                            for kerz in kernel_sizes:

                            
                                gin.bind_parameter("CNN_padding1.filter1", fl1)
                                gin.bind_parameter("CNN_padding1.filter2", fl2)
                                gin.bind_parameter("CNN_padding1.unit1", ut1)
                                gin.bind_parameter("CNN_padding1.unit2", ut2)
                                gin.bind_parameter("CNN_padding1.kernel_size", kerz)

                                gin.bind_parameter("trainloop.learning_rate", lr)

                                model = imagemodels.CNN_padding1()
                                model =  train_model.trainloop(
                                    model=model,
                                    metrics=[accuracy],
                                    train_dataloader=train_dataloader,
                                    test_dataloader=test_dataloader,
                                    train_steps=len(train_dataloader),
                                    eval_steps=150,
                                )

2022-12-06 11:27:38.219 | INFO     | src.data.data_tools:dir_add_timestamp:114 - Logging to ../../models/units_padding_all1_kernal3_l288/20221206-1127
100%|██████████| 1875/1875 [01:13<00:00, 25.41it/s]
2022-12-06 11:28:54.757 | INFO     | src.models.train_model:trainloop:171 - Epoch 0 train 0.5189 test 0.4561 metric ['0.8327']
100%|██████████| 1875/1875 [01:09<00:00, 26.97it/s]
2022-12-06 11:30:06.751 | INFO     | src.models.train_model:trainloop:171 - Epoch 1 train 0.3211 test 0.3187 metric ['0.8821']
100%|██████████| 1875/1875 [01:10<00:00, 26.68it/s]
2022-12-06 11:31:19.521 | INFO     | src.models.train_model:trainloop:171 - Epoch 2 train 0.2747 test 0.3180 metric ['0.8883']
100%|██████████| 1875/1875 [01:10<00:00, 26.60it/s]
2022-12-06 11:32:32.457 | INFO     | src.models.train_model:trainloop:171 - Epoch 3 train 0.2373 test 0.2668 metric ['0.9054']
100%|██████████| 1875/1875 [01:08<00:00, 27.27it/s]
2022-12-06 11:33:43.594 | INFO     | src.models.train_model:trainloop:171 - Epoch

In [5]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.


In [6]:
test 1

filter2 = [16,32,64,128,256]
unit1 = [64,128,256,512,1024]

resultaat optimaal:
filter2 = [128]
unit1 = [256]

test 2

filter2 = [64,..,..,..,..,128,..,..,256]
unit1 = [64,..,..,..,128,..,..,..,..,256]

resultaat optimaal:
filter2 = [168]
unit1 = [75]

test 3

filter2 = [64,..,..,..,..,128,..,..,256]
unit1 = [64,..,..,..,128,..,..,..,..,256]

resultaat optimaal:
filter2 = [168]
unit1 = [75]



SyntaxError: invalid syntax (3602976460.py, line 1)

Run the experiment, and study the result with tensorboard. 

Locally, it is easy to do that with VS code itself. On the server, you have to take these steps:

- in the terminal, navigate to ~/code/ML22 
- activate the python environment for the shell with `poetry shell`. Note how the correct environment is being activated.
- run `tensorboard --logdir=models` in the terminal
- tensorboard will launch at `localhost:6006` and vscode will notify you that the port is forwarded
- you can either press the `launch` button in VScode or open your local browser at `localhost:6006`



Experiment with things like:

- changing the amount of units1 and units2 to values between 16 and 1024. Use factors of 2: 16, 32, 64, etc.
- changing the batchsize to values between 4 and 128. Again, use factors of two.
- all your experiments are saved in the `models` directory, with a timestamp. Inside you find a saved_config.gin file, that 
contains all the settings for that experiment. The `events` file is what tensorboard will show.
- plot the result in a heatmap: units vs batchsize.
- changing the learningrate to values between 1e-2 and 1e-5 
- changing the optimizer from SGD to one of the other available algoritms at [torch](https://pytorch.org/docs/stable/optim.html) (scroll down for the algorithms)

A note on train_steps: this is a setting that determines how often you get an update. 
Because our complete dataset is 938 (60000 / 64) batches long, you will need 938 trainstep to cover the complete 60.000 images.

This can actually be a bit confusion, because every value below 938 changes the meaning of `epoch` slightly, because one epoch is no longer
the full dataset, but simply `trainstep` batches. Setting trainsteps to 100 means you need to wait twice as long before you get feedback on the performance,
as compared to trainsteps=50. You will also see that settings trainsteps to 100 improves the learning, but that is simply because the model has seen twice as 
much examples as compared to trainsteps=50.

This implies that it is not usefull to compare trainsteps=50 and trainsteps=100, because setting it to 100 will always be better.
Just pick an amount, and adjust your number of epochs accordingly.